I want to complete the task in two ways:
1.	At first, I want to try a classification approach to tackle this problem.
2.	I want to develop a segmentation approach to accomplish this task.

# Classification
We can divide the classification into two sections:
1.	Data preparation
2.	Train and validate my custom made model
3.	Test the dataset given by Extend AI
## Libraries selection
I can use PyTorch as I have good expertise in this deep learning library. However, to train the dataset, I used RAM-Net architecture which is built on Tensorflow and Keras. Therefore, I choose TensorFlow and Keras to tackle this problem.
Model Description
I developed this Residual Attention Mobilenet (RAM-Net) in the last year for medical image analysis, and this is a published work at ICMLA 2020 conference. This is a mobile-friendly computer vision network with fewer parameters and flops with the capabilities to achieve higher performance on image datasets. So, I decide to choose this model as I am confident enough about its performance.
## Dataset Preparation
I use the offline data augmentation technique to produce a total of 357 images (51*7 where 51 images from each image) from these real 7 images (which are provided by extend AI). I split this dataset by training and validation. Here I use 90% data for training and 10% data for validation. Furthermore, I use these real 7 images for testing.
Here, I try to perform a binary classification problem where I divide the dataset into two classes: non_problametic (those images which do not have any issues) and problematic (those images which have knots and spots)
In the test dataset, two images are regarded as non_problematic images, and five images are counted as problematic images. Therefore, in the augmented dataset, I have 102 images for the non_problematic class and the remaining 255 images for the problematic class.
Data Augmentation Functionality: rotation, width shift, height shift, shear, zoom, horizontal flip, and fill mode=nearest


In [1]:
#import libraries
import os
from glob import glob
from keras.initializers import glorot_uniform
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
from sklearn.metrics import classification_report
import functools
import keras
from keras.layers.normalization import BatchNormalization
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import roc_curve
import pandas as pd
import numpy as np
import keras.backend as K
import tensorflow as tf
from sklearn.metrics import roc_auc_score
import numpy
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical # convert to one-hot-encoding
from keras.callbacks import TensorBoard
from keras.preprocessing.image import ImageDataGenerator
from keras import layers
from keras import Model
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.applications import MobileNet
from keras.optimizers import Adam
from keras.callbacks import ReduceLROnPlateau, EarlyStopping,ModelCheckpoint
import random
from keras.layers import Dense, Dropout, GlobalAveragePooling2D,GlobalMaxPooling2D,DepthwiseConv2D,Concatenate
import matplotlib.pyplot as plt
import tensorflow
from at import augmented_conv2d
from sklearn.metrics import confusion_matrix
tensorboard = TensorBoard(log_dir='./logs', histogram_freq=0,
                          write_graph=True, write_images=False)

Using TensorFlow backend.


Resize the whole dataset to 224X224, then normalize the dataset, and at last convert it to a numpy array.

In [2]:


X_train = np.load("D:/polynomial/covid19/anomaly dataset/new_224_224_train.npy")
y_train = np.load("D:/polynomial/covid19/anomaly dataset/new_train_labels.npy")
X_val = np.load("D:/polynomial/covid19/anomaly dataset/new_224_224_val.npy")

y_val = np.load("D:/polynomial/covid19/anomaly dataset/new_val_labels.npy")

print('train set: ', X_train.shape, y_train.shape )
print('validation set: ',X_val.shape, y_val.shape)

y_train = to_categorical(y_train)
y_val = to_categorical(y_val)
print(y_train.shape)
print(y_val.shape)
#print(y_val)

train set:  (321, 224, 224, 3) (321,)
validation set:  (36, 224, 224, 3) (36,)
(321, 2)
(36, 2)


Used Pre-trained weight from ImageNet

In [3]:
f=augmented_conv2d #attention augmented convolution
print(f,type(f))

#take pretrained weight of imagenet for thr last few classifier layers
pre_trained_model = MobileNet(input_shape=(224,224, 3), include_top=False, weights="imagenet")

<function augmented_conv2d at 0x00000251FFB17EA0> <class 'function'>


In [4]:
for layer in pre_trained_model.layers:
    #print(layer.name)
    layer.trainable = False

In [5]:
#print(len(pre_trained_model.layers))
last_layer = pre_trained_model.get_layer('conv_pw_13_relu')
#print('last layer output shape:', last_layer.output_shape)
last_output = last_layer.output
x=GlobalAveragePooling2D()(last_output)
#x=Dense(512,activation='relu')(x) #we add dense layers so that the model can learn more complex functions and classify for better results.
#x = Dropout(0.5)(x)
x = layers.Dense(512, activation='relu')(x)
# Add a dropout rate of 0.7
x = layers.Dropout(0.5)(x)

x = layers.Dense(256, activation='relu')(x)
# Add a dropout rate of 0.7
x = layers.Dropout(0.5)(x)


#x = layers.Dense(128, activation='relu')(x)
# Add a dropout rate of 0.7
#x = layers.Dropout(0.5)(x)


# Add a final sigmoid layer for classification
x = layers.Dense(2, activation='softmax',name='visualized_layer')(x)

model=Model(pre_trained_model.input, x)

# Hyperparameters and Online Data Augmentations

Loss function: Binary cross-entropy 

Optimizer: Adam

In addition, I also use online data augmentation for the training.

Online data augmentation function: rotation, width shift, height shift, shear, zoom, and fill mode=nearest



In [6]:
# Configure and compile the model
'''
smooth = 1.
def dice_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)


def dice_coef_loss(y_true, y_pred):
    return 1-dice_coef(y_true, y_pred)
'''
#top3_acc = functools.partial(keras.metrics.top_k_categorical_accuracy, k=3)

#top3_acc.__name__ = 'top3_acc'

optimizer = Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=True)
model.compile(loss='binary_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy','top_k_categorical_accuracy'])

model.summary()


train_datagen = ImageDataGenerator(rotation_range=60, width_shift_range=0.2, height_shift_range=0.2,
                                   shear_range=0.2, zoom_range=0.2, fill_mode='nearest')

train_datagen.fit(X_train)

val_datagen = ImageDataGenerator()
val_datagen.fit(X_val)

batch_size = 32
epochs = 3
history = model.fit_generator(train_datagen.flow(X_train,y_train, batch_size=batch_size),
                              epochs = epochs, validation_data = val_datagen.flow(X_val, y_val),
                              verbose = 1, steps_per_epoch=(X_train.shape[0] // batch_size),
                              validation_steps=(X_val.shape[0] // batch_size))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 225, 225, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 32)      128       
__________

Epoch 1/3
10/10 [==============================] - 5s 498ms/step - loss: 0.9783 - acc: 0.5844 - top_k_categorical_accuracy: 1.0000 - val_loss: 0.5643 - val_acc: 0.7812 - val_top_k_categorical_accuracy: 1.0000
Epoch 2/3
10/10 [==============================] - 3s 331ms/step - loss: 0.8359 - acc: 0.6626 - top_k_categorical_accuracy: 1.0000 - val_loss: 1.8075 - val_acc: 0.2500 - val_top_k_categorical_accuracy: 1.0000
Epoch 3/3
10/10 [==============================] - 4s 353ms/step - loss: 0.5797 - acc: 0.7570 - top_k_categorical_accuracy: 1.0000 - val_loss: 0.6817 - val_acc: 0.7188 - val_top_k_categorical_accuracy: 1.0000


In [7]:
def identity_block(X, f, filters):
    """
    Implementation of the identity block as defined in Figure 3

    Arguments:
    X -- input tensor of shape (m, n_H_prev, n_W_prev, n_C_prev)
    f -- integer, specifying the shape of the middle CONV's window for the main path
    filters -- python list of integers, defining the number of filters in the CONV layers of the main path
    stage -- integer, used to name the layers, depending on their position in the network
    block -- string/character, used to name the layers, depending on their position in the network

    Returns:
    X -- output of the identity block, tensor of shape (n_H, n_W, n_C)
    """

    # defining name basis
    #conv_name_base = 'res' + str(stage) + block + '_branch'
    #bn_name_base = 'bn' + str(stage) + block + '_branch'

    # Retrieve Filters
    F1, F2, F3 = filters

    # Save the input value. You'll need this later to add back to the main path.
    X_shortcut = X
    #print(X_shortcut)

    # First component of main path
    X = Conv2D(filters=F1, kernel_size=(1, 1), strides=(1, 1), padding='valid',
               kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3)(X)
    X = Activation('relu')(X)
    #extra


    # Second component of main path (≈3 lines)
    X = Conv2D(filters=F2, kernel_size=(f, f), strides=(1, 1), padding='same',
               kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3)(X)
    X = Activation('relu')(X)



    # Third component of main path (≈2 lines)
    X = Conv2D(filters=F3, kernel_size=(1, 1), strides=(1, 1), padding='valid',
               kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3)(X)

    # Final step: Add shortcut value to main path, and pass it through a RELU activation (≈2 lines)
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)

    return X


def convolutional_block(X, f, filters, s=2):
    """
    Implementation of the convolutional block as defined in Figure 4

    Arguments:
    X -- input tensor of shape (m, n_H_prev, n_W_prev, n_C_prev)
    f -- integer, specifying the shape of the middle CONV's window for the main path
    filters -- python list of integers, defining the number of filters in the CONV layers of the main path
    stage -- integer, used to name the layers, depending on their position in the network
    block -- string/character, used to name the layers, depending on their position in the network
    s -- Integer, specifying the stride to be used

    Returns:
    X -- output of the convolutional block, tensor of shape (n_H, n_W, n_C)
    """

    # defining name basis
    #conv_name_base = 'res' + str(stage) + block + '_branch'
    #bn_name_base = 'bn' + str(stage) + block + '_branch'

    # Retrieve Filters
    F1, F2, F3 = filters

    # Save the input value
    X_shortcut = X

    ##### MAIN PATH #####
    # First component of main path
    X = Conv2D(F1, (1, 1), strides=(s, s),  kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3)(X)
    X = Activation('relu')(X)

    # Second component of main path (≈3 lines)
    X = Conv2D(filters=F2, kernel_size=(f, f), strides=(1, 1), padding='same',
               kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3)(X)
    X = Activation('relu')(X)



    # Third component of main path (≈2 lines)
    X = Conv2D(filters=F3, kernel_size=(1, 1), strides=(1, 1), padding='valid',
               kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3)(X)

    ##### SHORTCUT PATH #### (≈2 lines)
    X_shortcut = Conv2D(filters=F3, kernel_size=(1, 1), strides=(s, s), padding='valid',
                        kernel_initializer=glorot_uniform(seed=0))(X_shortcut)
    X_shortcut = BatchNormalization(axis=3)(X_shortcut)

    # Final step: Add shortcut value to main path, and pass it through a RELU activation (≈2 lines)
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)

    return X
'''for layer in model.layers[:20]:
    layer.trainable = False

for layer in model.layers[20:]:
    layer.trainable = True'''
for layer in model.layers[:12]:
    layer.trainable=True
for layer in model.layers[12:13]:
    layer.trainable=DepthwiseConv2D(64,strides=(2, 2),dilation_rate=(1,1))
for layer in model.layers[13:25]:
    layer.trainable=True
for layer in model.layers[25:26]:
    layer.trainable=DepthwiseConv2D(128,strides=(2, 2),dilation_rate=(1,1))
for layer in model.layers[26:38]:
    layer.trainable=True
for layer in model.layers[38:39]:
    layer.trainable=DepthwiseConv2D(256,strides=(2, 2),dilation_rate=(2,2))

for layer in model.layers[39:75]:
    layer.trainable=True
    #layer.trainable=identity_block(layer.output,3,[64, 64, 256], stage=1, block='a')
for layer in model.layers[75:76]:
    layer.trainable=DepthwiseConv2D(512,strides=(2, 2),dilation_rate=(2,2))
'''
for layer in model.layers[76:81]:
    layer.trainable=identity_block(layer.output,3,[128, 128, 512], stage=2, block='a')
    #layer.trainable=True'''
for layer in model.layers[76:78]:
    ca1=convolutional_block(layer.output, f = 3, filters = [64, 64, 256],s=1)
    a1=identity_block(ca1, 3, [64, 64, 256])
    b1=identity_block(a1,3,[64, 64, 256])
    #ba1=BatchNormalization()(b1)
    ac1= augmented_conv2d(b1, 512)
    ca2 = convolutional_block(ac1, f=3, filters=[128, 128, 512], s=1)
    a2 = identity_block(ca2, 3, [128, 128, 512])
    b2 = identity_block(a2, 3, [128, 128, 512])
    c2 = identity_block(b2, 3, [128, 128, 512])
    #ba2=BatchNormalization()(c2)
    ac2=augmented_conv2d(c2,512)
    ca3= convolutional_block(ac2, f=3, filters=[256, 256, 1024], s=1)
    a3 = identity_block(ca3, 3, [1256, 256, 1024])
    b3 = identity_block(a3, 3, [256, 256, 1024])
    c3 = identity_block(b3, 3, [256, 256, 1024])




    #layer.trainable=Concatenate([ca,a,b])

    layer.trainable=c3
    #print('man')
    layer.trainable=True
for layer in model.layers[78:81]:
    layer.trainable=True
for layer in model.layers[81:82]:

    #res2=identity_block(res1,3,[128, 128, 512], stage=3, block='b')
    #res3=identity_block(res2,3,[128, 128, 512], stage=3, block='c')

    a=DepthwiseConv2D(1024, strides=(2, 2),dilation_rate=(4,4))
    b=DepthwiseConv2D(1024, strides=(2, 2),dilation_rate=(8,8))
    c=DepthwiseConv2D(1024, strides=(2, 2),dilation_rate=(16,16))
    d=Concatenate([a,b])
    layer.trainable=Concatenate([d,c])
for layer in model.layers[82:]:
    layer.trainable=True

## Hyperparameter

epoch= 40 
batch size=32
learning rate=0.0001

In [17]:
def auroc(y_true, y_pred):
    return tf.py_func(roc_auc_score, (y_true, y_pred), tf.double)


optimizer = Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
model.compile(loss='binary_crossentropy',
              optimizer=optimizer,
              metrics=['acc','top_k_categorical_accuracy'])

#save_best = ModelCheckpoint(filepath='D:/polynomial/covid19/inception_dl4_val_new/checkpoint-{val_acc:.4f}.h5',monitor='val_acc',mode='auto',save_best_only=True)
save_best=ModelCheckpoint('D:/polynomial/covid19/anomaly dataset/RAM_Net.h5', verbose=1,monitor='val_acc',save_best_only=True, save_weights_only=True)
learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', patience=5, verbose=1, factor=np.sqrt(0.1),
                                            min_lr=0.5e-6, cooldown=0,mode='auto')
callbacks=[learning_rate_reduction,tensorboard,save_best]

model.summary()

batch_size = 32
epochs = 40
history = model.fit_generator(train_datagen.flow(X_train,y_train, batch_size=batch_size),
                              epochs = epochs, validation_data = val_datagen.flow(X_val, y_val),
                              verbose = 1, steps_per_epoch=(X_train.shape[0] // batch_size),
                              validation_steps=(X_val.shape[0] // batch_size),
                              callbacks=callbacks)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 225, 225, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 32)      128       
__________

Epoch 1/40
10/10 [==============================] - 8s 772ms/step - loss: 0.2429 - acc: 0.8906 - top_k_categorical_accuracy: 1.0000 - val_loss: 0.0200 - val_acc: 1.0000 - val_top_k_categorical_accuracy: 1.0000

Epoch 00001: val_acc improved from -inf to 1.00000, saving model to D:/polynomial/covid19/anomaly dataset/RAM_Net.h5
Epoch 2/40
10/10 [==============================] - 4s 376ms/step - loss: 0.3146 - acc: 0.8520 - top_k_categorical_accuracy: 1.0000 - val_loss: 0.0533 - val_acc: 1.0000 - val_top_k_categorical_accuracy: 1.0000

Epoch 00002: val_acc did not improve from 1.00000
Epoch 3/40
10/10 [==============================] - 4s 357ms/step - loss: 0.1054 - acc: 0.9653 - top_k_categorical_accuracy: 1.0000 - val_loss: 0.0123 - val_acc: 1.0000 - val_top_k_categorical_accuracy: 1.0000

Epoch 00003: val_acc did not improve from 1.00000
Epoch 4/40
10/10 [==============================] - 4s 355ms/step - loss: 0.0772 - acc: 0.9811 - top_k_categorical_accuracy: 1.0000 - val_loss: 0.0017

# Performance evaluation

For the performance evaluation, I use several performance evaluation matrices:
1. Test accuracy
2. confusion matrix
3. Precision
4. Recall
5. F1 Score


In [18]:
model.load_weights('D:/polynomial/covid19/anomaly dataset/RAM_Net.h5')
loss_val, acc_val,top5 = model.evaluate(X_val, y_val, verbose=1)
print("Validation accuracy= %f  ;  loss_v= %f" % (acc_val, loss_val))
#print('top5 validate accuracy: ',top5)
#print('top3 validate accuracy: ',top3)


X_test = np.load("D:/polynomial/covid19/anomaly dataset/test.npy")
y_test = np.load("D:/polynomial/covid19/anomaly dataset/test_labels.npy")
#print("1st ytest: ",y_test)
print("True Label for the test set: ",y_test)
target_names = ['Non_Problematic', 'Problematic']
#print("1st y_test",y_test, y_test.shape)
y_pred = model.predict(X_test)
#print("Test Accuracy",y_pred.astype(float),y_pred.shape,type(y_pred))
np.set_printoptions(precision=3, suppress=True)
#print("1: ",y_pred)
np.set_printoptions(formatter={'float': '{: 0.3f}'.format})
#print("2: ",y_pred)

y_classes = y_pred.argmax(axis=-1)
print("Predicted label for the test set:",y_classes)

#print("5th y_pred",y_classes,y_classes.shape)
print(classification_report(y_test, y_classes, target_names=target_names))
y_test = to_categorical(y_test)
#print("2nd y_test",y_test)

#print("2nd ytest: ",y_test)

loss_test, acc_test,top5 = model.evaluate(X_test, y_test, verbose=1)
print("Test: accuracy = %f  ;  loss = %f" % (acc_test, loss_test))
#print("top5 test accuracy: ",top5)
#print("top3 test accuracy: ",top3)


pre=model.predict(X_test,verbose=1)
#print("pre",pre)
n_values=3
c = np.eye(n_values, dtype=int)[np.argmax(pre, axis=1)]
#print('one-hot-encoding',c)
cm = confusion_matrix(y_test.argmax(axis=1), c.argmax(axis=1))
print("confusion matrix",cm)
cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
#print('new_cm',cm)
#print('acc',cm.diagonal())

36/36 [==============================] - 0s 3ms/step
Validation accuracy= 1.000000  ;  loss_v= 0.031549
True Label for the test set:  [1 0 1 1 1 1 0]
Predicted label for the test set: [1 0 1 1 1 1 0]
                 precision    recall  f1-score   support

Non_Problematic       1.00      1.00      1.00         2
    Problematic       1.00      1.00      1.00         5

       accuracy                           1.00         7
      macro avg       1.00      1.00      1.00         7
   weighted avg       1.00      1.00      1.00         7

7/7 [==============================] - 0s 4ms/step
Test: accuracy = 1.000000  ;  loss = 0.088632
7/7 [==============================] - 0s 4ms/step
confusion matrix [[2 0]
 [0 5]]


#### To sum up, all the evaluation functions prove that My algorithm (RAM-Net) perfectly detects the problematic and non_problematic images with 100% test accuracy.

In the Segmentation_model_extend_ai.ipynb I provide the segmentation related solution for this same problem. 
